# Abstract

## 1. Program Objectives
In this program, we faced a Credit risk control problem. To efficiently detect the potential default customers, we build a scor-card model(A/B/C/F).
## 2. Process
Firstly, we use rolling rate analysis and vintage analysis to determine the bad label and the behavior window length. Secondly, at the part 'Explore and Preprocess', we preprocess the raw data and then use WOE binning methodology to make model more robust. At the next step, IV and Random Forest were used to select vital features. Finally, we build a Logistic Regression model and get the scores card. Also, metrics such as AUC, KS, PSI were used to evaluate the behavior of our model.
## 3. Data Source
https://www.lendingclub.com/personal-banking

Data from LendingClub contains one-year-loan details(2017 Q3 to 2018 Q2).

Rolling rate analysis and vintage analysis requires panel data, but we only get cross-section data. The detail steps of rolling rate analysis and vintage analysis are not difficult.

For rolling rate analysis, we should determine the performance period, observation point and the observation period first. Then we calculate M(n) in each period sequentially. Finally, find the max conversion rate and its label(M(n)) in performance period as bad label.

For vintage analysis, we should plot M(n) series grouped by the loan date. In dim of time, we select the minimum date as observation window length which M(n) doesn't change too much, and in dim of loan date, we can analyse the quality of loan on each loan date.

In [60]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency


from sklearn.model_selection import train_test_split, KFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Config

In [2]:
DATA_DIR = './data'
TARGET = 'label'

### Data

In [3]:
data1 = pd.read_csv(f'{DATA_DIR}/LoanStats_2017Q3.csv', header=1, skipfooter=2, engine='python')
data2 = pd.read_csv(f'{DATA_DIR}/LoanStats_2017Q4.csv', header=1, skipfooter=2, engine='python')
data3 = pd.read_csv(f'{DATA_DIR}/LoanStats_2018Q1.csv', header=1, skipfooter=2, engine='python')
data4 = pd.read_csv(f'{DATA_DIR}/LoanStats_2018Q2.csv', header=1, skipfooter=2, engine='python')

In [103]:
data = pd.concat([data1, data2, data3, data4], axis=0)
data = data.query("loan_status in ['Fully Paid', 'Late (16-30 days)', 'Late (31-120 days)']")
data[TARGET] = np.where(data['loan_status'] == 'Fully Paid', 0, 1)

In [171]:
train, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data[TARGET])

print(f'Columns n is {len(data.columns)}')
print(f'Train n is {len(train)}, Test n is {len(test)}')

Columns n is 146
Train n is 49918, Test n is 12480


### Explore and Preprocess

In [172]:
# Drop columns whose Nan quantity above 80%

def dropNaColumns(train, test):
    NaN = train.isnull().sum()/train.shape[0]
    remain_col = NaN[NaN<=0.8].index
    return train[remain_col], test[remain_col]

train, test = dropNaColumns(train, test)

f'Remaining {train.shape[1]} columns.'

'Remaining 106 columns.'

In [173]:
# Drop numerical columns whose coef of var below 10%
# Drop categorical columns whose nunique below 2

def dropConsColumns(train, test):
    NUM = train.select_dtypes(exclude='object').columns
    CAT = train.select_dtypes(include='object').columns

    cv = train[NUM].std()/train[NUM].mean()
    nunique = train[CAT].nunique()

    remain_col = cv[cv>0.1].index.tolist() + nunique[nunique>=2].index.tolist()

    return train[remain_col], test[remain_col]

train, test = dropConsColumns(train, test)
f'Remaining {train.shape[1]} columns.'

'Remaining 102 columns.'

In [174]:
# Cate -> Num transform
# 'emp_length': Ordinary category, transform to ordered number
# 'grade': Ordinary category, transform to ordered number
# 'sub_grade': Ordinary category, transform to ordered number
# 'zip_code': Three bits long, the first bit represent the state, containing the same information of feature 'addr_state'
# 'term': Contain number information
# 'int_rate': Contain number information
# 'revol_util': Contain number information
# 'issue_d', 'last_pymnt_d', 'earliest_cr_line', 'last_credit_pull_d': Datetime, transform to the gap between the date and '2018-12'


def cate2Num(train, test):
    target_date = pd.to_datetime('2018-12')

    for dataset in [train, test]:
        dataset['emp_length'] = dataset['emp_length'].map({
            np.nan: 0,
            '< 1 year': 1,
            '1 year': 2,
            '2 years': 3,
            '3 years': 4,
            '4 years': 5,
            '5 years': 6,
            '6 years': 7,
            '7 years': 8,
            '8 years': 9,
            '9 years': 10,
            '10+ years': 11,
        })
        dataset['grade'] = dataset['grade'].map({
            np.nan: 0,
            'A':1,
            'B':2,
            'C':3,
            'D':4,
            'E':5,
            'F':6,
            'G':7,
        })
        dataset['sub_grade'] = dataset['sub_grade'].str[1:].astype(int)
        dataset['zip_code'] = dataset['zip_code'].str[1:3].astype(int)
        dataset['term'] = dataset['term'].str.extract(r'(\d+)').astype('Int64')
        dataset['int_rate'] = dataset['int_rate'].str.extract(r'(\d+.\d+)').astype('float')
        dataset['revol_util'] = dataset['revol_util'].str.extract(r'(\d+.\d+)').astype('float')

        for dt in ['issue_d', 'last_pymnt_d', 'earliest_cr_line', 'last_credit_pull_d']:
            dataset[dt] = pd.to_datetime(dataset[dt], format='%b-%Y')
            dataset[dt] = (target_date.year - dataset[dt].dt.year) * 12 + (target_date.month - dataset[dt].dt.month)

    return train, test

train, test = cate2Num(train, test)
f'Remaining {train.shape[1]} columns.'

'Remaining 102 columns.'

In [175]:
# Fill NaN

ColumnNanCnt = train.isnull().sum(axis=0)
ColumnNanRatio = ColumnNanCnt/len(train)
NanCols = ColumnNanRatio[ColumnNanRatio>0].index.tolist()
NanCols

['dti',
 'mths_since_last_delinq',
 'mths_since_last_major_derog',
 'mths_since_rcnt_il',
 'il_util',
 'all_util',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'mo_sin_old_il_acct',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_tl_120dpd_2m',
 'percent_bc_gt_75',
 'emp_title',
 'revol_util',
 'last_pymnt_d']

In [176]:
def fillNa(train, test):
    fill_0 = ['mo_sin_old_il_acct', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'emp_length']
    fill_mean = ['dti', 'il_util', 'all_util', 'avg_cur_bal','bc_open_to_buy','bc_util', 'num_tl_120dpd_2m', 'percent_bc_gt_75', 'revol_util']
    fill_999 = ['mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_rcnt_il', 'mths_since_recent_revol_delinq', 'last_pymnt_d']

    for dataset in [train, test]:
        for col in fill_0:
            dataset[col] = dataset[col].fillna(0)
        for col in fill_mean:
            dataset[col] = dataset[col].fillna(dataset[col].mean())
        for col in fill_999:
            dataset[col] = dataset[col].fillna(dataset[col].max()+1)
    return train, test

train, test = fillNa(train, test)
f'Remaining {train.shape[1]} columns.'

'Remaining 102 columns.'

In [177]:
# Drop categorical columns
print(train.select_dtypes(include='object').nunique())

def dropCateColumns(train, test):
    for dataset in [train, test]:
        dataset.drop('loan_status', inplace=True, axis=1)
        dataset.drop('emp_title', inplace=True, axis=1) # too many categories
    return train, test

train, test = dropCateColumns(train, test)
f'Remaining {train.shape[1]} columns.'

emp_title               20725
home_ownership              4
verification_status         3
loan_status                 3
pymnt_plan                  2
purpose                    12
title                      12
addr_state                 50
initial_list_status         2
application_type            2
hardship_flag               2
disbursement_method         2
debt_settlement_flag        2
dtype: int64


'Remaining 100 columns.'

In [178]:
f'After preprocessing, {train.shape[1]-1} columns are retained.'

'After preprocessing, 99 columns are retained.'

### Feature Selection

In [179]:
pos_n = np.sum(train[TARGET])
neg_n = train.shape[0] - pos_n

In [180]:
train.reset_index(drop=True, inplace=True)

In [181]:
# IV selection

# WOE bins
class WOE(BaseEstimator, TransformerMixin):

    def __init__(self, cols, max_bins=10, min_samples=0.05, cv=5):
        self.max_bins = max_bins
        self.min_samples = min_samples
        self.cv = cv
        self.iv_ = {}
        self.mapping_ = {}
        self.retain_ = []
        self.cols = cols

    def calc_chi2(self, row1, row2):
        table = [[row1['good']+1, row1['bad']+1],
                 [row2['good']+1, row2['bad']+1]]
        return chi2_contingency(table, correction=False)[0]

    def merge_bins(self, bins_stats):
        # 自底向上合并
        while len(bins_stats) > self.max_bins:
            # 计算相邻箱的卡方值
            chi2_values = []
            for i in range(len(bins_stats) - 1):
                chi2 = self.calc_chi2(bins_stats.iloc[i], bins_stats.iloc[i+1])
                chi2_values.append(chi2)

            # 找到最小卡方值对应的箱
            min_idx = np.argmin(chi2_values)

            # 合并箱
            bins_stats.iloc[min_idx]['total'] += bins_stats.iloc[min_idx+1]['total']
            bins_stats.iloc[min_idx]['good'] += bins_stats.iloc[min_idx+1]['good']
            bins_stats.iloc[min_idx]['bad'] += bins_stats.iloc[min_idx+1]['bad']
            bins_stats.iloc[min_idx]['max_val'] = bins_stats.iloc[min_idx+1]['max_val']
            bins_stats.iloc[min_idx]['bad_rate'] = bins_stats.iloc[min_idx]['bad'] / bins_stats.iloc[min_idx]['total']

            # 删除被合并的箱
            bins_stats = bins_stats.drop(min_idx+1).reset_index(drop=True)

            # 检查最小样本量
            min_sample_count = int(len(data) * self.min_samples)
            if bins_stats['total'].min() < min_sample_count:
                break

        return bins_stats


    def fit(self, X):
        eps = 1e-10
        for col in self.cols:
            data = X[[col, TARGET]].sort_values(col)
            if data[col].dtype == 'object' or data[col].nunique() <= self.max_bins:
                grouped = data.groupby(col).agg({
                    TARGET: ['count', 'sum']
                })
                grouped.columns = ['total', 'bad']
                grouped['good'] = grouped['total'] - grouped['bad']
                grouped['bad_pct'] = grouped['bad'] / pos_n
                grouped['good_pct'] = grouped['good'] / neg_n
                grouped['woe'] = np.log((grouped['bad_pct']+eps) / (grouped['good_pct'] + eps))
                grouped['iv'] = (grouped['bad_pct'] - grouped['good_pct']) * grouped['woe']

                iv = grouped['iv'].sum()
                if iv > 0.1:
                    self.retain_.append(f'woe_{col}')
                self.iv_[col] = iv
                self.mapping_[col] = grouped['woe']

            else:
                # 1. init bins
                n_init = min(50, len(data)//10)
                data['init_bin'] = pd.qcut(data[col], q=n_init, duplicates='drop')
                 # 2. stats in each bins
                bins_stats = data.groupby('init_bin').agg(
                    total=(TARGET, 'count'),
                    good=(TARGET, lambda x: (x == 0).sum()),
                    bad=(TARGET, lambda x: (x == 1).sum()),
                    min_val=(col, 'min'),
                    max_val=(col, 'max')
                ).reset_index()
                bins_stats = bins_stats[bins_stats['total'] > 0].reset_index(drop=True)

                # 3. bins merge
                bins_stats = self.merge_bins(bins_stats)

                # 4. bins bounds
                bins_stats['cut_points'] = list(bins_stats['max_val'].iloc[:-1]) + [np.inf]
                bins_stats['bad_pct'] = bins_stats['bad'] / pos_n
                bins_stats['good_pct'] = bins_stats['good'] / neg_n
                bins_stats['woe'] = np.log((bins_stats['bad_pct']+eps) / (bins_stats['good_pct']+ eps))
                bins_stats['iv'] = (bins_stats['bad_pct'] - bins_stats['good_pct']) * bins_stats['woe']

                iv = bins_stats['iv'].sum()
                if iv > 0.1:
                    self.retain_.append(f'woe_{col}')
                self.iv_[col] = iv
                self.mapping_[col] = bins_stats[['cut_points', 'woe']]

    def transform(self, X):
        res = pd.DataFrame(index=X.index)
        for col in self.cols:
            MAP = self.mapping_[col]
            if type(MAP) == pd.Series:
                res[f'woe_{col}'] = X[col].map(MAP)
            else:
                cut_points = [-np.inf] + list(MAP['cut_points'])
                kk = pd.cut(X[col], cut_points, labels=False)
                woe_mapping = {i: woe for i, woe in enumerate(MAP['woe'])}
                kk = np.array([woe_mapping.get(x) for x in kk])
                res[f'woe_{col}'] = kk
        return res


    def fit_transform(self, X):
        self.fit(X)
        eps = 1e-10

        res = pd.DataFrame(columns=[f'woe_{col}' for col in self.cols], index=X.index, dtype=np.float64)

        kf = KFold(n_splits=self.cv, shuffle=True, random_state=42)
        for train_idx, val_idx in kf.split(X):
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            for col in self.cols:
                if X[col].dtype == 'object' or X[col].nunique() <= self.max_bins:
                    grouped = X_train.groupby(col).agg({
                        TARGET: ['count', 'sum']
                    })
                    grouped.columns = ['total', 'bad']
                    grouped['good'] = grouped['total'] - grouped['bad']
                    grouped['bad_pct'] = grouped['bad'] / pos_n
                    grouped['good_pct'] = grouped['good'] / neg_n
                    grouped['woe'] = np.log((grouped['bad_pct']+eps) / (grouped['good_pct']+eps))
                    res.loc[X_val.index, f'woe_{col}'] = X_val[col].map(grouped['woe'])
                else:
                    data = X_train[[col, TARGET]].sort_values(col)
                    n_init = min(50, len(data)//10)
                    data['init_bin'] = pd.qcut(data[col], q=n_init, duplicates='drop')

                    bins_stats = data.groupby('init_bin').agg(
                        total=(TARGET, 'count'),
                        good=(TARGET, lambda x: (x == 0).sum()),
                        bad=(TARGET, lambda x: (x == 1).sum()),
                        min_val=(col, 'min'),
                        max_val=(col, 'max')
                    ).reset_index()
                    bins_stats = bins_stats[bins_stats['total'] > 0].reset_index(drop=True)

                    bins_stats = self.merge_bins(bins_stats)

                    cut_points = [-np.inf] + list(bins_stats['max_val'].iloc[:-1]) + [np.inf]
                    bins_stats['bad_pct'] = bins_stats['bad'] / pos_n
                    bins_stats['good_pct'] = bins_stats['good'] / neg_n
                    bins_stats['woe'] = np.log((bins_stats['bad_pct']+eps) / (bins_stats['good_pct']+eps))
                    kk = pd.cut(X_val[col], cut_points, labels=False)
                    woe_mapping = {i: woe for i, woe in enumerate(bins_stats['woe'])}
                    kk = np.array([woe_mapping.get(x) for x in kk])
                    res.loc[X_val.index, f'woe_{col}'] = kk

        return res


woe = WOE([col for col in train.columns if col != TARGET])
train_woe = woe.fit_transform(train)
print(f'{len(woe.retain_)} columns were retained.')

20 columns were retained.


In [182]:
train = pd.concat([train, train_woe[woe.retain_]], axis=1)
test_woe = woe.transform(test)
test = pd.concat([test, test_woe[woe.retain_]], axis=1)

In [183]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [184]:
# Random Forest selection

from sklearn.ensemble import RandomForestClassifier

# Attain feature importance according to Random Forest
rf = RandomForestClassifier()
rf.fit(train[woe.retain_], train[TARGET])

# Sort according to feature importance
feature_importance = pd.DataFrame({
    'feature': woe.retain_,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

# Filter features
features = feature_importance.iloc[:len(woe.retain_)//2]['feature']
features

4           woe_out_prncp
5       woe_out_prncp_inv
11    woe_last_pymnt_amnt
8     woe_total_rec_prncp
6         woe_total_pymnt
7     woe_total_pymnt_inv
3         woe_installment
1         woe_funded_amnt
2     woe_funded_amnt_inv
0           woe_loan_amnt
Name: feature, dtype: object

### Logistic Regression

In [185]:
# LR forward

from sklearn.linear_model import LogisticRegression

cv = 5
threshold = 0.01
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
selected_col = []
remain_col = list(features)
current_auc = 0.5

while remain_col:
    best_auc = 0.5
    best_feature = None
    best_test_pred = None
    for col in remain_col:
        columns = selected_col + [col]
        pred = pd.Series(index=train.index, dtype=int)
        test_pred = np.zeros(len(test))
        for train_idx, val_idx in kf.split(train):
            X_train, y_train = train.iloc[train_idx][columns], train.iloc[train_idx][TARGET]
            X_val = train.iloc[val_idx][columns]

            lr = LogisticRegression(class_weight='balanced')
            lr.fit(X_train, y_train)

            pred.iloc[val_idx] = lr.predict_proba(X_val)[:, 1]
            test_pred += lr.predict_proba(test[columns])[:, 1]

        auc = roc_auc_score(train[TARGET], pred)
        if auc > best_auc:
            best_auc = auc
            best_feature = col
            best_test_pred = test_pred/cv
    if best_auc - current_auc > 0.005:
        current_auc = best_auc
        selected_col.append(best_feature)
        remain_col.remove(best_feature)
        print(f'Epoch {len(selected_col)}: {best_feature} in. Valid dataset AUC={best_auc},', end = ' ')
        test_auc = roc_auc_score(test[TARGET], best_test_pred)
        print(f'Test dataset AUC={test_auc}.')
    else:
        break

Epoch 1: woe_last_pymnt_amnt in. Valid dataset AUC=0.9753063820638704, Test dataset AUC=0.974517443408607.
Epoch 2: woe_out_prncp in. Valid dataset AUC=0.9938043664569471, Test dataset AUC=0.994793717339426.


In [201]:
# Examine the coefficients' logic (>0)

lr = LogisticRegression(class_weight='balanced')
lr.fit(train[selected_col], train[TARGET])

for coef in lr.coef_[0]:
    if coef < 0:
        print('Logic Error: coef < 0.')
        break
else:
    print('Logic examination pass.')
lr.coef_

Logic examination pass.


array([[0.61913119, 0.77073966]])

### Score Card

In [206]:
def get_card(cols):
    score_card = pd.DataFrame()
    for j, col in enumerate(cols):
        col = col[4:]
        card = woe.mapping_[col].copy()
        if type(card) == pd.Series:
            card.index.name = 'cut_points'
            card.reset_index(inplace=True)
            card.insert(0, 'group', range(len(card)))
        else:
            card['cut_points'] = card['cut_points'].astype(str)
            for i in range(card.shape[0]-1, -1, -1):
                if i == card.shape[0]-1:
                    card.loc[i, 'cut_points'] = '>' + card.loc[i-1, 'cut_points']
                elif i == 0:
                    card.loc[i, 'cut_points'] = '<' + card.loc[i, 'cut_points']
                else:
                    card.loc[i, 'cut_points'] = '(' + card.loc[i-1, 'cut_points'] + ', ' + card.loc[i, 'cut_points'] + ']'

            card.index.name =  'group'
            card.reset_index(inplace=True)
        card.insert(0, 'feature', [col]*len(card))
        card['coef'] = [lr.coef_[0][j]]*len(card)
        card['score'] = card['coef']*card['woe']

        score_card = score_card._append(card, ignore_index=True)
    return score_card

score_card = get_card(selected_col)
score_card

,feature,group,cut_points,woe,coef,score
0,last_pymnt_amnt,0,<34.22,-0.114551,0.619131,-0.070922
1,last_pymnt_amnt,1,"(34.22, 108.25]",0.934875,0.619131,0.578811
2,last_pymnt_amnt,2,"(108.25, 184.35]",2.276994,0.619131,1.409758
3,last_pymnt_amnt,3,"(184.35, 257.56]",2.827470,0.619131,1.750575
4,last_pymnt_amnt,4,"(257.56, 324.65]",3.192544,0.619131,1.976604
5,last_pymnt_amnt,5,"(324.65, 388.58]",3.889355,0.619131,2.408021
6,last_pymnt_amnt,6,"(388.58, 486.98]",3.383129,0.619131,2.094601
7,last_pymnt_amnt,7,"(486.98, 588.54]",3.625517,0.619131,2.244671
8,last_pymnt_amnt,8,"(588.54, 733.46]",3.191368,0.619131,1.975875
9,last_pymnt_amnt,9,"(733.46, 908.5]",2.725680,0.619131,1.687553


In [246]:
BASEPOINTS = 600
BASEODDS = 0.05
PDO = 10

B = PDO/np.log(2)
A = BASEPOINTS + PDO/np.log(2)*np.log(BASEODDS)

f'A={A} B={B}'

'A=556.7807190511264 B=14.426950408889635'

### Model Evaluation

In [244]:
# KS (max(TPR-FPR))
p_test = lr.predict_proba(test[selected_col])[:, 1]
score = A + B * np.log(p_test/(1-p_test))
sum_bad = np.sum(test[TARGET])
sum_good = len(test) - sum_bad
cum_bad = 0
cum_good = 0
max_gap = 0

a = np.max(score)
b = np.min(score)
bin = (a-b)/10
print('bins\t\t\t\tgood\tbad\tcum_good\tcum_bad\tabs(diff)')
for i in range(1, 11):
    s1 = b + (i-1)*bin
    s2 = b + i*bin
    total = np.sum((score >= s1) & (score < s2))
    bad = np.where((score >= s1) & (score < s2), test[TARGET], 0).sum()
    good = total - bad
    cum_bad += bad
    cum_good += good
    max_gap = max(max_gap, abs(cum_good/sum_good - cum_bad/sum_bad))
    print(f'[{s1:.02f}, {s2:.02f})\t{good/sum_good:.02%}\t{bad/sum_bad:.02%}\t{cum_good/sum_good:.02%}\t{cum_bad/sum_bad:.02%}\t{abs(cum_good/sum_good - cum_bad/sum_bad):.02}')

bins				good	bad	cum_good	cum_bad	abs(diff)
[367.15, 413.67)	44.13%	0.05%	44.13%	0.05%	0.44
[413.67, 460.18)	0.00%	0.00%	44.13%	0.05%	0.44
[460.18, 506.70)	39.00%	0.10%	83.13%	0.15%	0.83
[506.70, 553.21)	11.75%	2.97%	94.88%	3.12%	0.92
[553.21, 599.73)	5.12%	6.86%	100.00%	9.98%	0.9
[599.73, 646.24)	0.00%	0.05%	100.00%	10.04%	0.9
[646.24, 692.75)	0.00%	0.00%	100.00%	10.04%	0.9
[692.75, 739.27)	0.00%	0.00%	100.00%	10.04%	0.9
[739.27, 785.78)	0.00%	0.36%	100.00%	10.39%	0.9
[785.78, 832.30)	0.00%	84.23%	100.00%	94.62%	0.054


In [250]:
# PSI (sum{(A-B)*ln(A/B)})
p_test = lr.predict_proba(test[selected_col])[:, 1]
p_train = lr.predict_proba(train[selected_col])[:, 1]
test_score = A + B * np.log(p_test/(1-p_test))
train_score = A + B * np.log(p_train/(1-p_train))
a = max(np.max(test_score), np.max(train_score))
b = min(np.min(test_score), np.min(train_score))
test_n = len(test)
train_n = len(train)
bins = 10
gap = (a-b)/bins
PSI = 0

for i in range(1, bins+1):
    s1 = b + (i-1)*gap
    s2 = b + i*gap
    A_ = np.sum((test_score >= s1) & (test_score < s2))/test_n
    B_ = np.sum((train_score >= s1) & (train_score < s2))/train_n

    PSI += (A_-B_)*np.log((A_+1)/(B_+1))

print(f'PSI={PSI}')

PSI=0.0031432919968940195
